<a href="https://colab.research.google.com/github/takatakamanbou/AdvML/blob/2024/AdvML2024_ex05notebookA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AdvML ex05notebookA

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/AdvML/AdvML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?AdvML)




今回の話は，学部の科目「機械学習I」でも出てきています（復習しつつ一歩先へ進む感じ）．受講していないひとは，以下をどうぞ．

- 2024年度「機械学習I」 第4回 https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2024#ex04
- 2024年度「機械学習I」 第5回 https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2024#ex05


----
## 準備
----


In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc  # アニメーションのため
import pandas as pd
import seaborn
seaborn.set()

---
## ロジスティック回帰
---


---
### 2クラス識別のロジスティック回帰


$D$ 次元のデータ $\mathbf{x} = (x_1, x_2, \ldots, x_D)$ を2つのクラス $C_1, C_2$ に分類する問題を考える．学習データは，次の形で与えられるとする

$$
(\mathbf{x}_1, y_1), (\mathbf{x}_2, y_2), (\mathbf{x}_N, y_N)
$$

$\mathbf{x}_n \in {\cal R}^{D}$ に対して，$y_n \in \{0, 1\}$ はこのデータの所属クラスの正解を表す値である． $y_n = 1$ ならば $\mathbf{x}_n$ は $C_1$ に属し，$y_n = 0$ ならば $\mathbf{x}_n$ は $C_2$ に属すものとする（$n=1,2,\ldots,N$）．




#### モデル

ロジスティック回帰では，$\mathbf{x}$ が与えられたときにそれがクラス $C_1$ に属する確率 $p(C_1|\mathbf{x})$ を，次のようにモデル化する．

$$
\begin{aligned}
f(\mathbf{x}) &= \sigma(w_0 + w_1x_1+\cdots + w_Dx_D) = \sigma\left(w_0 + \sum_{d=1}^{D}w_dx_d \right) \\
&= \frac{1}{1+\exp{\left( - \left( w_0 + \sum_{d=1}^{D}w_dx_d \right) \right)}}
\end{aligned}
$$

パラメータは，$w_0, w_1, \ldots, w_D$ の $(D+1)$ 個ある．



関数 $\sigma(s)$ は **シグモイド関数** （ロジスティックシグモイド関数, sigmoid function, logistic sigmoid function）と呼ばれるものである．

$$
\sigma(s) = \frac{1}{1+\exp{(-s)}}\qquad (1)
$$

式から明らかなように，任意の実数 $s$ に対して $ 0 < \sigma(s) < 1$ となる．この性質のため，ロジスティクス回帰モデルの出力は確率の値として解釈できる（$p(C_1|\mathbf{x}) =  f(\mathbf{x})$, $p(C_2|\mathbf{x}) = 1- f(\mathbf{x})$）．

In [ ]:
# シグモイド関数の値を計算
xmin, xmax = -6, 6
X = np.linspace(xmin, xmax, num=100)
Y = 1/(1+np.exp(-X))

# グラフに描く
fig = plt.figure(facecolor='white')
ax = fig.add_subplot(111)
ax.set_xlim(xmin, xmax)
ax.set_ylim(-0.1, 1.1)
ax.axhline(y=0, color='black', linestyle='-')
ax.axvline(x=0, color='black', linestyle='-')
ax.axhline(y=1, color='gray', linestyle='--')
ax.plot(X, Y, linewidth=2)
ax.set_xlabel('$s$')
ax.set_ylabel('$\sigma(s)$')
#ax.legend()
plt.show()

#### 交差エントロピー

$f(\mathbf{x})$ を確率モデルとみなすと，上記の学習データに対する尤度は

$$
\prod_{n=1}^{N} f(\mathbf{x}_n)^{y_n} \cdot (1 - f(\mathbf{x}_n))^{1-y_n}
$$

と表される．ロジスティック回帰モデルの学習では，この尤度の最大化を考える．
ただし，この式の形のままでは扱いが難しいので，対数尤度

$$
\sum_{n=1}^{N} ( y_n f(\mathbf{x}_n) + (1 - y_n) (1 - f(\mathbf{x}_n)))
$$

の最大化を考える．この対数尤度に負号を付けた

$$
H = -\sum_{n=1}^{N} ( y_n f(\mathbf{x}_n) + (1 - y_n) (1 - f(\mathbf{x}_n)))
$$

という量は，情報理論において **交差エントロピー** (cross entropy) と呼ばれるものである．ロジスティック回帰モデルの学習は，対数尤度の最大化ともいえるし，交差エントロピーの最小化ともいえる．


線形回帰モデルの場合，モデルの出力と正解の値との間の二乗誤差 $E$ を最小化するパラメータ $\mathbf{w}$ は， $\frac{\partial E}{\partial \mathbf{w}} = \mathbf{0}$ とおいて得られる連立方程式を解くことで求まっていた．しかし，ロジスティック回帰モデルの交差エントロピー最小化の場合は，そのように簡単には解が求まらない．パラメータの初期値を適当に定めて，その値を修正して徐々に目的関数を最小化していく，逐次最適化を行う必要がある．交差エントロピーの最小化に用いることのできる最適化手法はいくつかあるが，ここでは **勾配法**，特に， **最急降下法** による解法を説明する．

#### 最急降下法による学習のアルゴリズム

最急降下法では $H$ の $\mathbf{w}$ に関する勾配

$$
\nabla{H} = \left( \frac{\partial H}{\partial w_0}, \frac{\partial H}{\partial w_1}, \ldots, \frac{\partial H}{\partial w_D}\right)
$$

が必要となるので，$\frac{\partial H}{\partial w_d}$ ($d=0,1,\ldots,D$) を求めよう．

ここで，$\hat{y}_n = f(\mathbf{x}_n)$ および

$$
\ell_n = y_n\log \hat{y}_n + (1-y_n)\log(1-\hat{y}_n) \qquad (3)
$$

とおくことにする．このとき，

$$
\frac{\partial H}{\partial w_d} = -\sum_{n=1}^{N}\frac{\partial \ell_n}{\partial w_d}
$$

である．

以下は，板書 + 演習の形で説明します．

#### デモ: 2次元2クラス識別問題

In [ ]:
## 2次元正規分布で2クラスのデータを生成する関数

def getData(seed=None):

    if seed != None:
        np.random.seed( seed )

    # two 2-D spherical Gaussians
    X0 = 1.0*np.random.randn(200, 2) + [3.0, 3.0]
    X1 = 1.0*np.random.randn(200, 2) + [7.0, 6.0]
    X  = np.vstack((X0, X1))
    lab0 = np.zeros(X0.shape[0], dtype=int)
    lab1 = np.zeros(X1.shape[0], dtype=int) + 1
    label = np.hstack((lab0, lab1))

    return X, label

# データの準備
X, lab = getData(seed=0)
N, D = X.shape
Yt = lab
X = np.vstack((np.ones(N), X.T)).T
print(f'データ数 N = {N}, 次元数 D = {D}')

In [ ]:
# モデル出力の計算
def model(X, w):
    return 1.0 / (1.0 + np.exp(-(X @ w)))

# 交差エントロピーと正解数
def score(Y, Yt):
    ce = -np.sum(Yt*np.log(Y)+(1.0-Yt)*np.log(1.0-Y)) # 交差エントロピー
    count = np.sum((Y >= 0.5)*Yt) + np.sum((Y < 0.5)*(1 - Yt)) # 正解数
    return ce, count

# 勾配の計算
def grad(X, Y, Yt):
    return (Y - Yt) @ X

In [ ]:
# パラメータの初期化
w = (np.random.random(D+1) - 0.5) * 0.2 # [-0.1, 0.1) の一様乱数

# 学習係数と学習繰り返し回数
eta = 0.2/N
nitr = 1000

fig = plt.figure(facecolor='white', figsize=(12, 6))
ax1 = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122)
elevation = 20
azimuth = -70
ax1.view_init(elevation, azimuth)
ax1.set_xlim(0, 10)
ax1.set_ylim(0, 10)
ax1.set_zlim(0, 1)
ax1.scatter(X[Yt==0, 1], X[Yt==0, 2], 0)
ax1.scatter(X[Yt==1, 1], X[Yt==1, 2], 1)
#fig.show()

ax2.set_xlim(0, nitr)
ax2.set_ylim(0, 300)

aList = []
xx, yy = np.meshgrid(np.linspace(0, 10, num=16), np.linspace(0, 10, num=16))
xxr, yyr = xx.ravel(), yy.ravel()
XX = np.vstack((np.ones(xxr.shape[0]), xxr, yyr)).T

iList = []
ceList = []

for i in range(nitr+1):

    Y = model(X, w)     # モデル出力の計算
    ce, count = score(Y, Yt) # 交差エントロピーと正解数の計算
    dw = grad(X, Y, Yt) # 勾配の計算
    w -= eta * dw       # パラメータの更新

    if (i < 100 and i % 10 == 0) or i % 100 == 0:
        iList.append(i)
        ceList.append(ce)
        ZZ = model(XX, w)
        zz = ZZ.reshape(xx.shape)
        a1 = ax1.plot_wireframe(xx, yy, zz, color='green')
        a2 = ax2.plot(iList, ceList, color='blue', marker='.')
        rr = count/N*100
        s = f'H = {ce:.3f}\nrate = {rr:.1f}%'
        a3 = ax2.text(500, 240, s, size=20)
        aList.append([a1]+a2 + [a3])

anim = animation.ArtistAnimation(fig, aList, interval=300)
rc('animation', html='jshtml')
plt.close()
anim


---
### 多クラス識別のロジスティック回帰


**［ロジスティック回帰の問題設定（$K$クラスの場合）］**

$D$次元のデータを$K$個のクラス $C_1, C_2, \ldots, C_K$ に識別するモデルを学習させる．学習データは $N$ 個あり，次のように与えられる．

$$
(\mathbf{x}_1, \mathbf{y}_1), (\mathbf{x}_2, \mathbf{y}_2),\ldots , (\mathbf{x}_N, \mathbf{y}_N)
$$

ただし，$\mathbf{x}_n \in {\cal R}^{D}$ はモデルへの入力である．また，$\mathbf{y}_n \in \{0, 1\}^{K}$ （$0$か$1$のみを要素にもつ$K$次元ベクトル）はこのデータの所属クラスの正解を表す値である（$n=1,2,\ldots,N$）．

$\mathbf{y}_n = (y_{n,1}, y_{n,2}, \ldots, y_{n,K})$ の要素 $y_{n,k}$ は，$n$番目の学習データが $k$ 番目のクラスに所属するならば $1$，さもなくば $0$ をとる．したがって，$\mathbf{y}_n$ の要素はどれか一つだけが $1$ で他は全て $0$ である．

学習モデルは次式で定める．
$$
\begin{aligned}
\hat{y}_k &= \frac{\exp s_k}{\displaystyle\sum_{k=1}^{K}\exp{s_k}} \qquad (k = 1, 2, \ldots, K) \qquad (*)\\
s_k &= w_{k,0} + \sum_{d=1}^{D}w_{k,d}x_d
\end{aligned}
$$
このモデルのパラメータは $w_{k,d}$ ($k = 1, 2, \ldots, K, d = 0, 1, \ldots, D$) の $K\times (D+1)$ 個ある．

このとき，モデルの出力と正解の値との間の「遠さ」を，次式の交差エントロピーで定義する．
$$
\begin{aligned}
H &= -\sum_{n=1}^{N} \sum_{k=1}^{K} y_{n,k}\log{\hat{y}_{n,k}}
\end{aligned}
$$
この $H$ の値がなるべく小さくなるようにパラメータ $\{ w_{k,d} \}$ を求めたい．

式 $(*)$ は **ソフトマックス関数**（**softmax**関数）と呼ばれるものである．
式の形から分かるように，$0 < \hat{y}_k < 1$ および $\sum_{k=1}^{K}\hat{y}_k = 1$ となる性質がある．そのため，$\hat{y}_k$ は，$\mathbf{x}$ という値をもつデータがクラス $C_k$ に所属する確率 $p(C_k|\mathbf{x})$ を表すと解釈できる．

導出過程は省略するが，上記の交差エントロピのパラメータに関する勾配は次のようになる．

$$
\frac{\partial H}{\partial w_{k,d}} =  -\sum_{n}^{N}(y_{n,k} - \hat{y}_{n,k}) x_{n,d} \qquad(k = 1, 2, \ldots, K, d = 0, 1, \ldots, D)
$$


---
## 確率的勾配降下法(SGD)，バッチSGD
---

板書して解説します．

---
### 確率的勾配降下法(SGD)

---
### バッチSGD

---
### デモ: ロジスティック回帰による手書き数字の識別

In [ ]:
# 手書き数字データの入手
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/minimnist.npz
rv = np.load('minimnist.npz')
datL = rv['datL'].astype(float)
labL = rv['labL']
datT = rv['datT'].astype(float)
labT = rv['labT']
print(datL.shape, labL.shape, datT.shape, labT.shape)

K = 10 # クラス数

# 学習データの用意
NL, D = datL.shape # 学習データの数と次元数
XL = np.empty((NL, D+1))
XL[:, 0] = 1.0
XL[:, 1:] = datL/255
YtL = np.zeros((NL, K))
for ik in range(K):
    YtL[labL == ik, ik] = 1.0

# テストデータの用意
NT, _ = datT.shape # テストデータの数
XT = np.empty((NT, D+1))
XT[:, 0] = 1.0
XT[:, 1:] = datT/255
YtT = np.zeros((NT, K))
for ik in range(K):
    YtT[labT == ik, ik] = 1.0

In [ ]:
# パラメータの初期化
W = (np.random.random((K, D+1)) - 0.5) * 0.2 # [-0.1, 0.1) の一様乱数

# 学習係数と学習繰り返し回数
eta = 0.01
nitr = 20000

# 学習
for i in range(nitr+1):

    # 学習データの一つをランダムに選択
    n = np.random.randint(NL)
    x, yt = XL[n, :], YtL[n]
    # モデル出力の計算
    exps = np.exp(W @ x)
    y = exps / np.sum(exps)
    # 最急降下法
    dW = (y - yt)[:, np.newaxis] @ x[np.newaxis, :]
    W -= eta * dW

    if (i < 1000 and i % 100 == 0) or (i % 1000 == 0):
        # モデル出力の計算
        exps = np.exp(XL @ W.T)
        Y = exps / np.sum(exps, axis=1)[:, np.newaxis]
        # 交差エントロピー
        ce = -np.sum(YtL * np.log(Y))
        # 正解数
        count = np.sum(labL == np.argmax(Y, axis=1))
        print(f'{i}  {ce/NL:.3f}  {count/NL:.3f}')

print()

# テスト
exps = np.exp(XT @ W.T)
Y = exps / np.sum(exps, axis=1)[:, np.newaxis]
ce = -np.sum(YtT * np.log(Y))
count = np.sum(labT == np.argmax(Y, axis=1))
print(f'テスト: {ce/NT:.3f}  {count/NT:.3f}')